In [17]:
import numpy as np
import cvxopt as opt
from cvxopt import solvers
import pandas as pd


solvers.options['show_progress'] = False

# === Core Functions ===

def create_covariance_matrix(volatilities, correlations):
    vol_matrix = np.outer(volatilities, volatilities)
    return correlations * vol_matrix

def optimization(expected_returns, covariance_matrix, risk_tolerance=1.0, l2_penalty=0.1):
    n = len(expected_returns)
    P = opt.matrix(risk_tolerance * covariance_matrix + l2_penalty * np.eye(n))
    q = opt.matrix(-expected_returns)
    G = opt.matrix(-np.eye(n))
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    solution = solvers.qp(P, q, G, h, A, b)
    weights = np.array(solution['x']).flatten()
    port_return = np.dot(weights, expected_returns)
    port_risk = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return weights, port_return, port_risk



def adjust_inputs(predicted_returns, return_scores, predicted_vols, vol_scores, gamma=1.0, epsilon=1e-6):
    # Adjust returns based on interval score
    return_weights = 1 / (return_scores + epsilon)
    return_weights /= np.max(return_weights)
    adjusted_returns = predicted_returns * return_weights

    # Adjust volatility based on interval score
    adjusted_vols = predicted_vols * (1 + gamma * vol_scores)
    return adjusted_returns, adjusted_vols

def run_portfolio_optimization_for_period(period, predicted_returns, return_scores, predicted_vols, vol_scores, correlation_matrix, asset_names, risk_free_rate=0.02):
    adjusted_returns, adjusted_vols = adjust_inputs(predicted_returns, return_scores, predicted_vols, vol_scores)
    covariance_matrix = create_covariance_matrix(adjusted_vols, correlation_matrix)
    weights, port_return, port_risk = optimization(adjusted_returns, covariance_matrix)
    sharpe_ratio = (port_return - risk_free_rate) / port_risk

    print(f"\n📅 Period: {period}")
    print("🔺 Portfolio Allocation:")
    for name, weight in zip(asset_names, weights):
        print(f"  {name}: {weight:.4f} ({weight * 100:.2f}%)")

    print("📊 Portfolio Performance:")
    print(f"  Expected Return: {port_return:.4f}")
    print(f"  Risk (Std Dev): {port_risk:.4f}")
    print(f"  Sharpe Ratio: {sharpe_ratio:.4f}")


In [ ]:
# Define asset names
asset_names = ['Bond', 'Equity', 'Golds']

# Period labels for
periods = ['Feb 2025', 'Jan 2025', 'Dec 2024', 'Nov 2024', 'Oct 2024', 'Sep 2024', 'Aug 2024']


# === Your Data (replace with real TFT output) ===
predicted_returns_all = [
    np.array([0.013002467, 0.010407536, 0.008076485]),
    np.array([0.015123859, 0.005140172, 0.007952606]),
    np.array([0.005690926, 0.0000720,   0.007838867]),
    np.array([0.001442978, 0.047873445, 0.007735349]),
    np.array([0.007916870, 0.035785865, 0.007641279]),
    np.array([0.007425121, 0.043546267, 0.007555522]),
    np.array([0.009982087, 0.071743070, 0.007476744])
]

# Predicted volatilities for each asset class (Bond, Equity, Gold) per period:
# Updated Predicted volatilities for each asset class (Bond, Equity, Gold) per period:
predicted_vols_all = [
    np.array([0.002778549, 0.007113004, 0.008352804]),
    np.array([0.002777523, 0.009589466, 0.008574622]),
    np.array([0.003511720, 0.007590341, 0.011733534]),
    np.array([0.002876988, 0.008241563, 0.008897853]),
    np.array([0.002912198, 0.007614691, 0.008919279]),
    np.array([0.003371432, 0.008868512, 0.007796175]),
    np.array([0.003260702, 0.009087290, 0.010919048])
]




# Interval scores for returns (constant over periods,based on validation data set)
return_scores_all = [
    np.array([0.0582, 0.2376, 0.1787]) for _ in periods
]

# Interval scores for volatility (constant over periods,based on validation data set)
vol_scores_all = [
    np.array([0.0041, 0.0308, 0.0178]) for _ in periods
]















# Correlation matrices per period
# Rolling 12 month correlation between asset classes 
correlation_matrices = [
    np.array([
        [1.0, 0.831417315, 0.657522566],
        [0.831417315, 1.0, 0.898037934],
        [0.657522566, 0.898037934, 1.0]
    ]),
    np.array([
        [1.0, 0.86216755, 0.731770549],
        [0.86216755, 1.0, 0.910824279],
        [0.731770549, 0.910824279, 1.0]
    ]),
    np.array([
        [1.0, 0.859826515, 0.755568556],
        [0.859826515, 1.0, 0.898420078],
        [0.755568556, 0.898420078, 1.0]
    ]),
    np.array([
        [1.0, 0.7754664, 0.7206893],
        [0.7754664, 1.0, 0.907584704],
        [0.7206893, 0.907584704, 1.0]
    ]),
    np.array([
        [1.0, 0.740041169, 0.715919357],
        [0.740041169, 1.0, 0.921002238],
        [0.715919357, 0.921002238, 1.0]
    ]),
    np.array([
        [1.0, 0.788019312, 0.748599291],
        [0.788019312, 1.0, 0.912333484],
        [0.748599291, 0.912333484, 1.0]
    ]),
    np.array([
        [1.0, 0.829757457, 0.75721439],
        [0.829757457, 1.0, 0.908323839],
        [0.75721439, 0.908323839, 1.0]
    ])
]


# Store all results here
portfolio_results = []

for i in range(len(periods)):
    adjusted_returns, adjusted_vols = adjust_inputs(
        predicted_returns_all[i],
        return_scores_all[i],
        predicted_vols_all[i],
        vol_scores_all[i]
    )

    covariance_matrix = create_covariance_matrix(adjusted_vols, correlation_matrices[i])

    weights, port_return, port_risk = optimization(
        adjusted_returns,
        covariance_matrix,
        risk_tolerance=1.0,
        l2_penalty=0.1  # Regularization for diversification
    )

    sharpe_ratio = (port_return - 0.02) / port_risk  # Assuming 2% risk-free rate

    print(f"\n📅 Period: {periods[i]}")
    print("🔺 Portfolio Allocation:")
    for j, name in enumerate(asset_names):
        print(f"  {name}: {weights[j]:.4f} ({weights[j] * 100:.2f}%)")

    print("📊 Portfolio Performance:")
    print(f"  Expected Return: {port_return:.4f}")
    print(f"  Risk (Std Dev): {port_risk:.4f}")
    print(f"  Sharpe Ratio: {sharpe_ratio:.4f}")

    # Save results
    portfolio_results.append({
        "Period": periods[i],
        f"{asset_names[0]} Weight": weights[0],
        f"{asset_names[1]} Weight": weights[1],
        f"{asset_names[2]} Weight": weights[2],
        "Expected Return": port_return,
        "Risk (Std Dev)": port_risk,
        "Sharpe Ratio": sharpe_ratio
    })

# Convert to DataFrame
df_results = pd.DataFrame(portfolio_results)

# Save to CSV
df_results.to_csv("mvo_portfolio_results.csv", index=False)

print("\n✅ Portfolio results saved to mvo_portfolio_results.csv")



📅 Period: Feb 2025
🔺 Portfolio Allocation:
  Bond: 0.4029 (40.29%)
  Equity: 0.2981 (29.81%)
  Golds: 0.2989 (29.89%)
📊 Portfolio Performance:
  Expected Return: 0.0068
  Risk (Std Dev): 0.0055
  Sharpe Ratio: -2.3998

📅 Period: Jan 2025
🔺 Portfolio Allocation:
  Bond: 0.4216 (42.16%)
  Equity: 0.2825 (28.25%)
  Golds: 0.2959 (29.59%)
📊 Portfolio Performance:
  Expected Return: 0.0075
  Risk (Std Dev): 0.0063
  Sharpe Ratio: -1.9995

📅 Period: Dec 2024
🔺 Portfolio Allocation:
  Bond: 0.3630 (36.30%)
  Equity: 0.3060 (30.60%)
  Golds: 0.3310 (33.10%)
📊 Portfolio Performance:
  Expected Return: 0.0029
  Risk (Std Dev): 0.0073
  Sharpe Ratio: -2.3484

📅 Period: Nov 2024
🔺 Portfolio Allocation:
  Bond: 0.2957 (29.57%)
  Equity: 0.3982 (39.82%)
  Golds: 0.3061 (30.61%)
📊 Portfolio Performance:
  Expected Return: 0.0059
  Risk (Std Dev): 0.0067
  Sharpe Ratio: -2.1122

📅 Period: Oct 2024
🔺 Portfolio Allocation:
  Bond: 0.3488 (34.88%)
  Equity: 0.3570 (35.70%)
  Golds: 0.2942 (29.42%)
📊 Por